In [1]:
workspace='file:/home/wei/GitHub/BDM-HW/data/'
data=sqlContext.read.format('csv')\
                    .option("header", "false")\
                    .option("inferschema", "true")\
                    .option("mode", "DROPMALFORMED")\
                    .load(workspace + 'kddcup.data.corrected')

In [47]:
data.show()

+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|_c4| _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|  0|tcp|http| SF|181|5450|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|   9|   9| 1.0| 0.0|0.11| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|  0|tcp|http| SF|239| 486|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|  19|  19| 1.0| 0.0|0.05

(1)

In [2]:
q1 = data.describe('_c0', '_c4', '_c5', '_c10')
q1.show()

+-------+-----------------+------------------+-----------------+--------------------+
|summary|              _c0|               _c4|              _c5|                _c10|
+-------+-----------------+------------------+-----------------+--------------------+
|  count|           494021|            494021|           494021|              494021|
|   mean|47.97930249928647|3025.6102959185946|868.5324247349809|1.518154086567170...|
| stddev|707.7464723053723| 988218.1010503978|33040.00125210254| 0.01551959690293324|
|    min|                0|                 0|                0|                   0|
|    max|            58329|         693375640|          5155468|                   5|
+-------+-----------------+------------------+-----------------+--------------------+



In [36]:
q1_data = data.select(['_c0', '_c4', '_c5', '_c10']).rdd.map(list).collect()
q1_data = sc.parallelize(q1_data)

mode = [q1_data.map(lambda x:(x[i], 1)).reduceByKey(add).collect()[0][0] for i in range(4)]
median = [q1_data.map(lambda x:x[i]).sortBy(lambda x:x, False).collect()[int(494021/2)] for i in range(4)]

print('mode:', mode)
print('median:', median)

mode: [0, 256, 2032, 0]
median: [0, 520, 0, 0]


(2)

In [49]:
from operator import add

q2_data = data.select(['_c1', '_c2', '_c3', '_c11', '_c41']).rdd.map(list).collect()
q2_data = sc.parallelize(q2_data)

result = [q2_data.map(lambda x:(x[i], 1)).reduceByKey(add).sortBy(lambda x:x[1], False).collect() for i in range(5)]

header = ['protocol_type', 'service', 'flag', 'logged_in', 'intrusion_type']
for i, h in enumerate(header):
    print(h+str(':'))
    print(result[i], '\n')

protocol_type:
[('icmp', 283602), ('tcp', 190065), ('udp', 20354)] 

service:
[('ecr_i', 281400), ('private', 110893), ('http', 64293), ('smtp', 9723), ('other', 7237), ('domain_u', 5863), ('ftp_data', 4721), ('eco_i', 1642), ('ftp', 798), ('finger', 670), ('urp_i', 538), ('telnet', 513), ('ntp_u', 380), ('auth', 328), ('pop_3', 202), ('time', 157), ('csnet_ns', 126), ('remote_job', 120), ('gopher', 117), ('imap4', 117), ('domain', 116), ('discard', 116), ('iso_tsap', 115), ('systat', 115), ('shell', 112), ('echo', 112), ('rje', 111), ('sql_net', 110), ('whois', 110), ('printer', 109), ('courier', 108), ('nntp', 108), ('mtp', 107), ('netbios_ssn', 107), ('sunrpc', 107), ('uucp_path', 106), ('vmnet', 106), ('klogin', 106), ('bgp', 106), ('uucp', 106), ('nnsp', 105), ('supdup', 105), ('ssh', 105), ('login', 104), ('hostnames', 104), ('daytime', 103), ('efs', 103), ('link', 102), ('netbios_ns', 102), ('pop_2', 101), ('ldap', 101), ('http_443', 99), ('netbios_dgm', 99), ('exec', 99), ('ksh

(3)

In [57]:
q3_data = data.select(['_c2', '_c41']).rdd.map(list).collect()
q3_data = sc.parallelize(q3_data)

intrusionType = q3_data.map(lambda x:x[1]).distinct().collect()
result = [q3_data.map(lambda x:((x[0], x[1]), 1)).reduceByKey(add).filter(lambda x:x[0][1] == intrusionType[i])\
.map(lambda x:(x[0][0], x[1])).sortBy(lambda x:x[1], False).collect() for i in range(len(intrusionType))]

for i, t in enumerate(intrusionType):
    print('intrusionType:', t)
    print(result[i])

intrusionType: loadmodule.
[('telnet', 5), ('ftp_data', 3), ('ftp', 1)]
intrusionType: neptune.
[('private', 101317), ('telnet', 197), ('http', 192), ('finger', 177), ('ftp_data', 170), ('csnet_ns', 123), ('smtp', 120), ('remote_job', 118), ('pop_3', 118), ('iso_tsap', 115), ('discard', 115), ('systat', 113), ('domain', 112), ('gopher', 112), ('shell', 111), ('echo', 111), ('sql_net', 109), ('auth', 108), ('rje', 108), ('courier', 107), ('printer', 107), ('whois', 107), ('netbios_ssn', 106), ('nntp', 106), ('klogin', 106), ('bgp', 106), ('nnsp', 105), ('uucp_path', 105), ('mtp', 105), ('imap4', 105), ('uucp', 104), ('sunrpc', 104), ('ftp', 104), ('time', 103), ('daytime', 102), ('ssh', 102), ('login', 102), ('hostnames', 102), ('efs', 102), ('supdup', 101), ('vmnet', 101), ('pop_2', 101), ('ldap', 101), ('netbios_ns', 101), ('link', 99), ('netbios_dgm', 99), ('exec', 99), ('http_443', 98), ('kshell', 98), ('name', 97), ('ctf', 96), ('netstat', 92), ('other', 91), ('Z39_50', 91)]
intrus